In [3]:
import numpy as np
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split

from tensorflow.keras.callbacks import ModelCheckpoint
from collections import Counter
from sklearn.metrics import f1_score,accuracy_score, precision_score, recall_score
import matplotlib.pyplot as plt
from sklearn.model_selection import StratifiedKFold
import pandas as pd
import numpy as np
import pandas as pd
from tensorflow.keras.models import Model
from tensorflow.keras.layers import LSTM, Activation, Dense, Dropout, Input, Embedding
from tensorflow.keras.optimizers import RMSprop
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing import sequence
from nltk.stem import PorterStemmer 
from nltk import tokenize
from tensorflow.keras import backend as K
import nltk
import gensim.models.keyedvectors as word2vec #need to use due to depreceated model
from nltk.tokenize import RegexpTokenizer
nltk.download('punkt')
import pandas as pd

[nltk_data] Downloading package punkt to /home/phat-ngu/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


# Load data

In [5]:
# Load data as a list of pairs of sentences and labels (0 for non-spam, 1 for spam)
data = []
with open('SMSSpamCollection') as f:
    for line in f:
        firstword = (line.split()[0])
        label = None
        if firstword == 'ham':
            label = 0
        else: 
            label = 1
        text = line.replace(firstword,'').strip()
        data.append([text,label])
# transform data to numpy array
data = pd.DataFrame(np.array(data),columns=['text','label'])

# LSTM

In [11]:
#Dividing the dataset into features and lables
X = data['text']
Y = data['label']


max_words = 3500
max_len = 50
tok = Tokenizer(num_words=max_words)
tok.fit_on_texts(X)
sequences = tok.texts_to_sequences(X)
sequences_matrix = sequence.pad_sequences(sequences,maxlen=max_len)
X_train,X_test,Y_train,Y_test=train_test_split(sequences_matrix,Y,test_size=0.15,random_state=4) 



In [18]:
def RNN():
    inputs = Input(name='inputs',shape=[max_len])
    layer = Embedding(max_words,50,input_length=max_len)(inputs)
    layer = LSTM(64)(layer)
    layer = Dense(256,name='FC1')(layer)
    layer = Activation('relu')(layer)
    layer = Dropout(0.25)(layer)
    layer = Dense(1,name='out_layer')(layer)
    layer = Activation('sigmoid')(layer)
    model = Model(inputs=inputs,outputs=layer)
    return model
lstm_model = RNN()
lstm_model.summary()

def recall_m(y_true, y_pred):
        true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
        possible_positives = K.sum(K.round(K.clip(y_true, 0, 1)))
        recall = true_positives / (possible_positives + K.epsilon())
        return recall

def precision_m(y_true, y_pred):
        true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
        predicted_positives = K.sum(K.round(K.clip(y_pred, 0, 1)))
        precision = true_positives / (predicted_positives + K.epsilon())
        return precision

def f1_m(y_true, y_pred):
    precision = precision_m(y_true, y_pred)
    recall = recall_m(y_true, y_pred)
    return 2*((precision*recall)/(precision+recall+K.epsilon()))

lstm_model.compile(loss='binary_crossentropy',optimizer=RMSprop(),metrics=['accuracy',recall_m,precision_m,f1_m])
lstm_model.load_weights('weights-9529-test-9615-best.h5')


Model: "model_4"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
inputs (InputLayer)          [(None, 50)]              0         
_________________________________________________________________
embedding_4 (Embedding)      (None, 50, 50)            175000    
_________________________________________________________________
lstm_4 (LSTM)                (None, 64)                29440     
_________________________________________________________________
FC1 (Dense)                  (None, 256)               16640     
_________________________________________________________________
activation_8 (Activation)    (None, 256)               0         
_________________________________________________________________
dropout_4 (Dropout)          (None, 256)               0         
_________________________________________________________________
out_layer (Dense)            (None, 1)                 257 

In [32]:
lstm_model.evaluate(X_test,np.asarray(Y_test))

32/1 [================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================] - 0s 169us/sample


UnimplementedError:  Cast string to float is not supported
	 [[node loss/activation_9_loss/Cast (defined at /home/phat-ngu/.conda/envs/tf2/lib/python3.7/site-packages/tensorflow_core/python/framework/ops.py:1751) ]] [Op:__inference_distributed_function_4833]

Function call stack:
distributed_function


In [33]:
np.asarray(Y_test)

array(['1', '0', '0', '0', '1', '0', '0', '0', '0', '0', '1', '0', '0',
       '1', '0', '0', '0', '0', '0', '1', '0', '0', '0', '0', '0', '1',
       '0', '0', '0', '0', '0', '0', '1', '1', '0', '0', '0', '1', '0',
       '0', '0', '0', '0', '0', '0', '0', '0', '1', '0', '0', '0', '0',
       '0', '0', '0', '0', '1', '0', '1', '0', '0', '0', '0', '0', '0',
       '0', '0', '1', '0', '0', '0', '0', '0', '0', '0', '1', '0', '0',
       '0', '1', '0', '0', '0', '0', '0', '0', '0', '1', '1', '0', '0',
       '0', '0', '0', '0', '0', '0', '0', '0', '0', '1', '1', '0', '1',
       '0', '0', '0', '0', '0', '0', '1', '0', '0', '0', '0', '0', '0',
       '0', '0', '0', '0', '1', '0', '0', '1', '0', '0', '0', '0', '0',
       '1', '0', '0', '0', '0', '0', '1', '0', '0', '0', '0', '0', '0',
       '0', '0', '0', '0', '0', '0', '0', '1', '0', '1', '0', '0', '0',
       '0', '0', '1', '0', '1', '0', '0', '0', '0', '1', '0', '0', '0',
       '0', '0', '1', '1', '0', '0', '0', '0', '0', '0', '0', '0